In [16]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import io

### Prep dataset

In [2]:
#ascii char mapping from emnist
ascii_map = open('emnist/emnist-byclass-mapping.txt', 'r').readlines()
ascii_map = dict([(int(l.rstrip().split(' ')[0]), int(l.rstrip().split(' ')[1])) for l in ascii_map])

#load in handwritten text dataset
df = pd.read_parquet('telkia_iam_handwritten_text.gzip')

In [41]:
def sample(n):
    return df['bytes'].sample(n).apply(lambda x: np.array(Image.open(io.BytesIO(x)))).tolist()

### Using simple RNN + CTC

In [42]:
s = sample(10)

In [ ]:
s[0].shape

In [54]:
#init params

y_preds = []
for i in range(s.shape[1]):
    

(128, 2237)